In [1]:
import sys
sys.path.append('../')
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import utils.delphesRootToPixelAvTrackList as utils
import uproot
import awkward as ak
import plotly.figure_factory as ff
import plotly.graph_objects as go

In [2]:
tree = "Delphes"
branches = ["Track.PID", "Track.Charge", "Track.PT", "Track.P", "Track.Eta", "Track.Phi", "Track.EtaOuter", "Track.PhiOuter", "Track.XOuter", "Track.YOuter","Track.ZOuter"]
pionPID = 211 # plus/minus 

file = "minbias_0.00_0.10_GeV.root"
with uproot.open(file) as f:
    # load the branches
    temp = {}
    for branch in branches:
        temp[branch] = np.array(ak.flatten(f[tree][branch].array()))
        
    # selection
    cut = (abs(temp["Track.PID"])==pionPID)

    # apply selection
    for branch in branches:
        temp[branch] = temp[branch][cut]

In [3]:

xOuter = temp["Track.XOuter"]
yOuter = temp["Track.YOuter"]
etaOuter = temp["Track.EtaOuter"]
phiOuter = temp["Track.PhiOuter"]
alpha, beta, yentry, gamma, xc, yc, xc_actual, yc_actual, R, nx, ny, nz = utils.ArrangeSensor(temp, True)

In [4]:
def create_cylinder(z0):
    # Define the cylinder parameters
    radius = 30
    num_points=100

    # Define the cylinder's parametric equations
    theta = np.linspace(0, 2 * np.pi, num_points)  # Angle around the z-axis
    z = np.linspace(z0-30, z0+30, num_points)  # Height of the cylinder
    theta, z = np.meshgrid(theta, z)  # Create a meshgrid for the parameters

    x = radius * np.cos(theta)  # X coordinates of the cylinder
    y = radius * np.sin(theta)  # Y coordinates of the cylinder

    return x, y, z

def generate_particle_track(radius, charge, xc, yc, vz, omega):
    num_points = 50
    
    # Generate the circle in the xy plane
    theta0=np.arcsin(-yc/radius)
    print(round(radius * np.cos(theta0) + xc))
    print(round(radius * np.sin(theta0) + yc))
    #if round(radius * np.cos(theta0) + xc) != 0 or round(radius * np.sin(theta) + yc) != 0:
    theta0=np.arccos(-xc/radius)
    print(round(radius * np.cos(theta0)+ xc))
    print(round(radius * np.sin(theta0) + yc))
    if charge<0:
        theta = np.linspace(theta0, theta0+np.pi/20, num_points)
        t = (theta-theta0)/omega
    else:
        theta = np.linspace(theta0, theta0-np.pi/20, num_points)
        t = (theta0-theta)/omega
    x = radius * np.cos(theta) + xc
    y = radius * np.sin(theta) + yc
    z = t*vz*1000

    d = np.rint(np.sqrt(x**2+y**2))
    for i in range(len(d)):
        if d[i] == 30:
            print(x[i],y[i])

    return x, y, z

In [5]:
def plot(index):
    # Create the 3D surface plot
    
    theta=2*np.arctan(np.exp(-temp["Track.EtaOuter"][index]))
    pz = temp["Track.P"][index]*np.cos(theta)
    gammaR = np.sqrt(1+(temp["Track.P"][index]*5.36e-19)**2/((2.488e-28)**2*(2.99e8)**2))
    vz = pz*5.36e-19/(gammaR*2.488e-28)
    omega = 1.6027e-19*3.8/(gammaR*2.488e-28)

    z0 = temp["Track.ZOuter"][index]
    # get info for plotting sensor
    x0=30*np.cos(gamma[index])
    y0=30*np.sin(gamma[index])
    # Calculate the distance in the x and y directions
    dx = 8 * np.cos(gamma[index]+np.pi/2)
    dy = 8 * np.sin(gamma[index]+np.pi/2)
    # Calculate the endpoints of the line segment
    x1, y1 = x0 - dx, y0 - dy
    x2, y2 = x0 + dx, y0 + dy
    z1, z2 = z0 + 8, z0 - 8

    x_track, y_track, z_track = generate_particle_track(R[index], temp["Track.Charge"][index], xc_actual[index], yc_actual[index], vz, omega)
    fig = go.Figure()
    fig.add_trace(go.Scatter3d(
        x=x_track,
        y=y_track,
        z=z_track,
        mode='lines',
        line=dict(color='blue', width=4)
    ))
    
    fig.add_trace(go.Cone(x=[x0], y=[y0-yentry[index]], z=[z0], u=[nx[index]*10], v=[ny[index]*10], w=[nz[index]*10],colorscale='Greens', sizemode="raw", showscale=False, anchor='tail'))

    print(temp["Track.XOuter"][index],temp["Track.YOuter"][index])
    
    fig.add_trace(go.Mesh3d(
        x=np.array([x1,x1,x2,x2,x1]), 
        y=np.array([y1,y1,y2,y2,y1]), 
        z=np.array([z1,z2,z2,z1,z1]),
        i=[0, 0],
        j=[1, 2],
        k=[2, 3],
        color='red',
        opacity=1    
    ))

    # Create cylinder data
    x, y, z = create_cylinder(z0)
    fig.add_trace(go.Surface(x=x, y=y, z=z, colorscale='Greys', opacity=0.6, showscale=False))

    # Set the layout
    fig.update_layout(
        scene=dict(
            xaxis = dict(nticks=4, range=[-50,50],),
            yaxis = dict(nticks=4, range=[-50,50],),
            zaxis = dict(nticks=4, range=[z0-50,z0+50],),
            xaxis_title='X Axis',
            yaxis_title='Y Axis',
            zaxis_title='Z Axis',
            aspectmode="cube"
        )
    )


    # Show the plot
    #fig.write_html('plot.html', auto_open=True)
    fig.show()

In [9]:
plot(3)

0
0
0
304
-24.488379 -17.329723
